In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %reset

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %run common_info.ipynb

## Реализуем извлечение важных предложений на всех уровнях документа.

In [ ]:
%run process_pdf_title.ipynb

In [ ]:
%run extract_headers_text_structure.ipynb

### Выделяем заголовки, которые присутствуют на каждом уровне иерархии.

In [ ]:
def splitHeadersByLevels(headersAndTextHierarchy, level, headersByLevels, prevLevelNumeration):
    if level + 1 not in headersByLevels:
        headersByLevels[level + 1] = []
    levelHeaders = []
    for levelNumeration, nestedHierarchy in headersAndTextHierarchy.items():
        numerationDelimiter = '' if prevLevelNumeration == '' else '.'
        delimiter = '' if prevLevelNumeration == '' and levelNumeration == '' else ' '
        headersByLevels[level + 1] += splitHeadersByLevels(nestedHierarchy[1],
                                                           level + 1,
                                                           headersByLevels,
                                                           prevLevelNumeration + numerationDelimiter + levelNumeration)
        if isinstance(nestedHierarchy[0], list):
            for headerIndex in range(0, nestedHierarchy[0]):
                levelHeaders.append(prevLevelNumeration + numerationDelimiter + levelNumeration + delimiter + nestedHierarchy[0][headerIndex]['header'])
        else:
            levelHeaders.append(prevLevelNumeration + numerationDelimiter + levelNumeration + delimiter + nestedHierarchy[0]['header'])
    return levelHeaders

In [ ]:
from pprint import pprint

draftHeadersByLevels = {}
level = 1
draftHeadersByLevels[1] = splitHeadersByLevels(headersAndTextHierarchy, level, draftHeadersByLevels, '')

headersByLevels = {}
for level in draftHeadersByLevels:
    if draftHeadersByLevels[level] != []:
        headersByLevels[level] = draftHeadersByLevels[level]
draftHeadersByLevels = None

if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersByLevels)

### Для каждого заголовка на каждом уровне иерархии определяем все вложенные заголовки.

In [ ]:
def extractHeaderNestedHeaders(headersAndTextHierarchy, requiredHeader, extract, prevLevelNumeration):
    if not headersAndTextHierarchy:
        return []
    nestedHeaders = []
    for levelNumeration, nestedHierarchy in headersAndTextHierarchy.items():
        numerationDelimiter = '' if prevLevelNumeration == '' else '.'
        delimiter = '' if prevLevelNumeration == '' and levelNumeration == '' else ' '
        levelNumeration = prevLevelNumeration + numerationDelimiter + levelNumeration
        if isinstance(nestedHierarchy[0]['header'], list):
            for headerIndex in range(0, nestedHierarchy[0]):
                header = levelNumeration + delimiter + nestedHierarchy[0][headerIndex]['header']
                nestedHeaders += extractHeaderNestedHeaders(nestedHierarchy[1], requiredHeader, False, levelNumeration)
        else:
            header = levelNumeration + delimiter + nestedHierarchy[0]['header']
            if extract:
                nestedHeaders += [header] + extractHeaderNestedHeaders(nestedHierarchy[1], requiredHeader, extract, levelNumeration)
            if not extract and header == requiredHeader:
                nestedHeaders += extractHeaderNestedHeaders(nestedHierarchy[1], requiredHeader, True, levelNumeration)
            nestedHeaders += extractHeaderNestedHeaders(nestedHierarchy[1], requiredHeader, False, levelNumeration)
    return nestedHeaders

In [ ]:
from collections import OrderedDict
from pprint import pprint

headersAndNestedHeadersByLevels = OrderedDict()
for level in headersByLevels:
    headersAndNestedHeadersByLevels[level] = []
    for header in headersByLevels[level]:
        headersAndNestedHeadersByLevels[level].append({
            'header': header, 
            'nestedHeaders': extractHeaderNestedHeaders(headersAndTextHierarchy, header, False, '')
        })
        
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndNestedHeadersByLevels)

### Выделяем текст, который соотвествует заголовкам на каждом уровне иерархии.

In [ ]:
def splitTextByHeadersAndLevels(headerPattern, nestedHeadersPatterns, headerPatternsAndText):
    text = ''
    for headerPatternAndText in headerPatternsAndText:
        if headerPatternAndText['headerPattern'] == headerPattern or \
           headerPatternAndText['headerPattern'] in nestedHeadersPatterns:
            text += headerPatternAndText['text'] + ' '
    return text

In [ ]:
from collections import OrderedDict
from pprint import pprint

headersAndTextByLevels = OrderedDict()
for level, headersAndNestedHeaders in headersAndNestedHeadersByLevels.items():
    headersAndTextByLevels[level] = []
    for headerAndNestedHeaders in headersAndNestedHeaders:
        headersAndTextByLevels[level].append({
            'header': headerAndNestedHeaders['header'],
            'text': splitTextByHeadersAndLevels(headerAndNestedHeaders['header'],
                                                headerAndNestedHeaders['nestedHeaders'],
                                                headerPatternsAndText)
        })
        
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndTextByLevels)

### Добавляем нулевой уровень иерархии — уровень документа.

In [ ]:
from collections import OrderedDict
from pprint import pprint

headersAndTextByLevelsCopy = headersAndTextByLevels
headersAndTextByLevels = OrderedDict()
headersAndTextByLevels[0] = []
headersAndTextByLevels[0].append({
    'header': titlePageDict['workTitle'],
    'text': ''
})
for headerAndText in headersAndTextByLevelsCopy[1]:
    headersAndTextByLevels[0][0]['text'] += headerAndText['text']
for level, headersAndText in headersAndTextByLevelsCopy.items():
    headersAndTextByLevels[level] = []
    for headerAndText in headersAndText:
        headersAndTextByLevels[level].append(headerAndText)
  
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndTextByLevels)

### Бъем текст на предложения и определяем коэффициент этого предложения в тексте в соответствии с эвристиками.

In [ ]:
introductoryWords = set(['итак', 'следовательно', 'значит', 'наоборот', 'напротив', 'далее', 'наконец', 'впрочем', 'например', 'главное', 'кстати'])
introductoryBigrams = set(['в заключение', 'заметим что', 'между прочим', 'в общем', 'в частности', 'прежде всего', 'кроме того', 'сверх того', 'стало быть', 'к примеру', 'таким образом'])
introductoryTrigrams = set(['с одной стороны', 'с другой стороны'])

discursiveMarkers = set(['причина', 'ведь', 'поэтому'])
discursiveBigrams = set(['потому что', 'быть причина', 'стать причина', 'являться причина'])

advantagesMarkers = set(['достоинство', 'недостаток', 'плюс', 'минус'])

introductionWords = set(['актуальность', 'актуальный'])
introductionBigrams = set(['цель работа'])

bulletListMarkers = set(['во-первых', 'во-вторых', 'в-третьих', 'в-четвертых', 'в-четвёртых'])
bulletMarkersPattern = 'во\-первых|во\-вторых|в\-третьих|в\-четвертых|в\-четвёртых'

imageReferencesMarkers = ['рис', 'рисунок']
imageReferencesPattern = 'рис|рисунок'
tableReferencesMarkers = ['таблица', 'табла', 'таб']
tableReferencesPattern = 'таблица|табла|таб'

In [ ]:
%run extract_possible_terms.ipynb

In [ ]:
from pyrutok import Token, Sentence, Tokenizer, GraphemTag, TokenType
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from collections import Counter
from pprint import pprint
import re
from string import punctuation

morphAnalyzer = MorphAnalyzer()
imageNumerations = set([caption['numeration'] for caption in captionsToImages])
tableNumerations = set([caption['numeration'] for caption in captionsToTables])

headersAndSentencesByLevels = {}
for level, headersAndText in headersAndTextByLevels.items():
    headersAndSentencesByLevels[level] = []
    for blockIndex in range(0, len(headersAndText)):
        headersAndSentencesByLevels[level].append({
            'header': headersAndText[blockIndex]['header'],
            'sentences': []
        })
        imageReferences = set()
        tableReferences = set()   
        for tempText in headersAndText[blockIndex]['text'].split('\n'):
            tempText = tempText.replace('т.д.', 'так далее').replace('т. д.', 'так далее')
            tempText = tempText.replace('т.п.', 'тому подобное.').replace('т. п.', 'тому подобное.')
            tempText = tempText.replace('т.е.', 'то есть').replace('т. е.', 'то есть').replace('т.е', 'то есть')
    #         tempText = tempText.replace('\n', '\n\n')
            tempText = tempText.replace(':\n', '?? ').replace(': \n', '?? ').replace('.\n', '!! ').replace('. \n', '!! ')
            tempText = tempText.replace(',\n', ', ').replace(', \n', ', ').replace('\n', '. ')
            tempText = tempText.replace('Рис.', 'Рис_').replace('рис.', 'рис_').replace('англ.', 'англ_').replace('см.', 'см_').replace('См.', 'См_')
            tempText = tempText.replace('Таб.', 'Таб_').replace('таб.', 'таб_').replace('Табл.', 'Табл_').replace('табл.', 'табл_')
            tempText = tempText.replace('\uf0b7', '•')
            for url in urls:
                tempText = tempText.replace(url, '*URL*')
            # многоточия негативно влияют на разделение предложений
            tempText = re.sub(r' *\. *\. *\. *', '___', tempText)
            # убираем ссылки на литературу из текста
            tempText = re.sub(r'\[(\d+(,? ?|\-?|–?))*\]', '', tempText)
            tempText = tempText.replace('  ', ' ').replace(').', '_!').replace('].', '_?')
            # почему-то точки приводят токенайзер в замешательство
            tempText = re.sub(r'(\d+)\.', r'\1}', tempText)
#             for us in unusualSymbols:
#                 tempText = tempText.replace(us, '')
            for sentence in Tokenizer(tempText):
                sentenceText = re.sub(r'(\d+)\}', r'\1.', sentence.as_text())
                sentenceText = sentenceText.replace('_!', ').').replace('_?', '].').replace('!!', '.').replace('!', '.').replace('??', ':').replace(':.', ':')
                sentenceText = re.sub(r'( *\.)+', '.', sentenceText)
                sentenceText = sentenceText.replace('Рис_', 'Рис.').replace('рис_', 'рис.').replace('англ_', 'англ.').replace('см_', 'см.').replace('См_', 'См.')
                sentenceText = sentenceText.replace('Таб_', 'Таб.').replace('таб_', 'таб.').replace('Табл_', 'Табл.').replace('табл_', 'табл.')
                sentenceText = sentenceText.replace('___', '...')

                sentenceImportance = 1
                sentenceImageReferences = set()
                sentenceTableReferences = set()

                for term in terms:
                    if term in sentenceText:
                        sentenceImportance = 1.1

                sentenceWords = re.sub(r'[' + re.escape(punctuation) + r']', '', sentenceText).lower().split(' ')
                sentenceBigrams = [sentenceWords[i] + ' ' + sentenceWords[i+1] for i in range(0, len(sentenceWords)-1)]
                sentenceTrigrams = [sentenceWords[i] + ' ' + sentenceWords[i+1] + ' ' + sentenceWords[i+2] 
                                    for i in range(0, len(sentenceWords)-2)]
    #             print(sentenceBigrams)
                # проверяем наличие вводных слов и обрезаем их, если они в начале предложения
                if set(sentenceWords).intersection(introductoryWords) or \
                   set(sentenceBigrams).intersection(introductoryBigrams) or \
                   set(sentenceTrigrams).intersection(introductoryTrigrams):
                    sentenceImportance = 1.1
                if len(sentenceWords) > 0 and sentenceWords[0] in introductoryWords:
                    sentenceText = sentenceText.split(' ', 1)[1]
                    sentenceText = sentenceText[0].upper() + sentenceText[1:]
                if len(sentenceBigrams) > 0 and sentenceBigrams[0] in introductoryBigrams:
                    sentenceText = sentenceText.split(' ', 2)[2]
                    sentenceText = sentenceText[0].upper() + sentenceText[1:]
                if len(sentenceTrigrams) > 0 and sentenceTrigrams[0] in introductoryTrigrams:
                    sentenceText = sentenceText.split(' ', 3)[3]
                    sentenceText = sentenceText[0].upper() + sentenceText[1:]

                sentenceTextTemp = sentenceText
                for us in unusualSymbols:
                    sentenceText = sentenceText.replace(us, '')
                sentenceWordsAndNumbersNF = []
                for innerSentence in Tokenizer(sentenceText):
                    for token in innerSentence:
                        if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                            word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                            sentenceWordsAndNumbersNF.append(word)
                        if token.get_token_type() == TokenType.NUMBER:
                            sentenceWordsAndNumbersNF.append(token.get_escaped_data())
                sentenceBigramsNF = [sentenceWordsAndNumbersNF[i] + ' ' + sentenceWordsAndNumbersNF[i+1] 
                                     for i in range(0, len(sentenceWordsAndNumbersNF)-1)]
                sentenceTrigramsNF = [sentenceWordsAndNumbersNF[i] + ' ' + sentenceWordsAndNumbersNF[i+1] + ' ' + sentenceWordsAndNumbersNF[i+2]
                                     for i in range(0, len(sentenceWordsAndNumbersNF)-2)]
                sentenceText = sentenceTextTemp
    #             print(sentenceWordsAndNumbersNF)
                # запоминаем ссылки на изображения
                for trigram in sentenceTrigramsNF:
                    imageReference = re.findall(r'^(' + imageReferencesPattern + r')\ \d+\ \d+$', trigram)
                    if imageReference and trigram.split(' ', 1)[1] in imageNumerations:
                        imageReferences.add(trigram.split(' ', 1)[1])
                        sentenceImageReferences.add(trigram.split(' ', 1)[1])
#                 for bigram in sentenceBigramsNF:
#                     imageReference = re.findall(r'^(' + imageReferencesPattern + r')\ \d+$', bigram)
#                     if imageReference and bigram.split(' ')[1] in imageNumerations:
#                         imageReferences.add(bigram.split(' ')[1])
#                         sentenceImageReferences.add(bigram.split(' ')[1])
#                     # на случай, если человек не подписывает таблицы как таблицы
#                     if imageReference and bigram.split(' ')[1] in imageNumerations:
#                         imageReferences.add(bigram.split(' ')[1])
#                         sentenceImageReferences.add(bigram.split(' ')[1])
                # запоминаем ссылки на таблицы
                for bigram in sentenceBigramsNF:
                    tableReference = re.findall(r'^(' + tableReferencesPattern + r')\ \d+$', bigram)
                    if tableReference and bigram.split(' ')[1] in tableNumerations:
                        tableReferences.add(bigram.split(' ')[1])
                        sentenceTableReferences.add(bigram.split(' ')[1])
                # проверяем наличие дискурсивных маркеров
                if set(sentenceWordsAndNumbersNF).intersection(discursiveMarkers) or \
                   set(sentenceBigramsNF).intersection(discursiveBigrams):
                    sentenceImportance = 1.1
                # проверяем маркеры, важные для введения
                if (set(sentenceWordsAndNumbersNF).intersection(introductionWords) or \
                    set(sentenceBigramsNF).intersection(introductionBigrams)) and \
                   'введение' in headersAndText[blockIndex]['header'].lower():
                    sentenceImportance = 1.3
                # проверяем возможность сделать предложение частью маркированных списков
    #             if set(sentenceWordsAndNumbersNF).intersection(bulletListMarkers):
    #                 sentenceText = re.sub(r'(^|,? ?[а-я]* ?)(' + bulletMarkersPattern + r'),', '\n•', sentenceText, flags=re.IGNORECASE)
    #                 # потому что неизвестно, насколько это важно
    #                 sentenceImportance = 1.31

    #                 if sentenceText[0] == '\n' and headersAndSentencesByLevels[level][blockIndex]['sentences']:
    #                     headersAndSentencesByLevels[level][blockIndex]['sentences'][-1]['sentence'] = \
    #                         headersAndSentencesByLevels[level][blockIndex]['sentences'][-1]['sentence'][-1] + ':'

    #                 for sentencePart in sentenceText.split('\n'):
    #                     delimiter = '\n' if sentencePart != sentenceText.split()[-1] else ''
    #                     if sentencePart and sentencePart[0] not in listMarkers and sentencePart[-1] != ':':
    #                         sentencePart += ':'
    #                     headersAndSentencesByLevels[level][blockIndex]['sentences'].append({
    #                         'sentence': sentencePart.replace(' ,', ',').replace(' .', '.').replace(' :', ':').replace(' ;', ';').strip(),
    #                         'importance': sentenceImportance,
    #                         'betweenBullets': False,
    #                         'bulletListPart': True,
    #                         'betweenNumbering': False,
    #                         'numberedListPart': False,
    #                         'imageReferences': sentenceImageReferences,
    #                         'tableReferences': sentenceTableReferences
    #                     })
    #                 continue

                if not sentenceText.strip() or len(sentenceText.split(' ')) < 2:
                    continue

                # пытаемся выбросить записи таблиц            
#                 if len([p.start() for p in re.finditer(r'' + '|'.join([re.escape(p) for p in punctuation]) + r'', sentenceText)]) > 20:
#                     sentenceImportance = 0

                # выбрасываем кривые предложения
                if sentenceText[0] in punctuation+'–':
                    sentenceImportance = 0
                    
                # выбрасываем cлишком длинные предложения
                if len(sentenceText) > 400:
                    sentenceImportance = 0

    #             print([p.start() for p in re.finditer(r'' + '|'.join([re.escape(p) for p in punctuation]) + r'', sentenceText)], sentenceText)

                headersAndSentencesByLevels[level][blockIndex]['sentences'].append({
                    'sentence': sentenceText.replace(' ,', ',').replace(' .', '.').replace(' :', ':').replace(' ;', ';').strip(),
                    'importance': sentenceImportance,
                    'betweenBullets': False,
                    'bulletListPart': False,
                    'betweenNumbering': False,
                    'numberedListPart': False,
                    'imageReferences': sentenceImageReferences,
                    'tableReferences': sentenceTableReferences
                })
                headersAndSentencesByLevels[level][blockIndex]['imageReferences'] = imageReferences
                headersAndSentencesByLevels[level][blockIndex]['tableReferences'] = tableReferences

if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

Добавляем значимости предложениям, которые находятся в общей для подразделов текстовой части.

In [ ]:
for level in range(1, len(headersAndNestedHeadersByLevels)):
    headersAndNestedHeaders = headersAndNestedHeadersByLevels[level]
    for blockIndex in range(0, len(headersAndNestedHeaders)):
        headerAndNestedHeaders = headersAndNestedHeaders[blockIndex]
        if headerAndNestedHeaders['nestedHeaders']:
            firstNestedHeader = headerAndNestedHeaders['nestedHeaders'][0]
            firstNestedHeaderNumeration = firstNestedHeader.split(' ')[0].split('.')
            firstNestedHeaderLevel = len(firstNestedHeaderNumeration)
            # это первое предложение одной из вложенных частей
            commonPartEndMarker = next(iter([headerAndSentences 
                                             for headerAndSentences in headersAndSentencesByLevels[firstNestedHeaderLevel] 
                                             if headerAndSentences['header'] == firstNestedHeader][0]['sentences']))['sentence']
#             print(commonPartEndMarker)
            sentenceIndex = 0
            sentences = headersAndSentencesByLevels[level][blockIndex]['sentences']
            while sentenceIndex != len(sentences) and sentences[sentenceIndex]['sentence'] != commonPartEndMarker:
                if sentences[sentenceIndex]['importance'] > 0.5:
                    sentences[sentenceIndex]['importance'] = 1.5
                sentenceIndex += 1

if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

Делаем безусловно важными предложения, которые изначально являлись частью маркированных списков.

In [ ]:
for level, headersAndSentences in headersAndSentencesByLevels.items():
    for headerAndSentences in headersAndSentences:
        betweenMarkers = False
        betweenNumbering = False
        lastMarkedSentenceIndex = None
        for sentenceIndex in range(0, len(headerAndSentences['sentences'])):
            sentence = headerAndSentences['sentences'][sentenceIndex]
            if (sentence['sentence'][0] in listMarkers or \
                (sentence['sentence'][0] not in listMarkers and \
                 sentence['sentence'][-1] == ':')) and \
               sentence['importance'] != 1.31:
                if sentence['importance'] > 0.5:
                    sentence['importance'] = 2
                sentence['bulletListPart'] = True
            # с нумерованными списками сложнее, так что выделяем их меньше 
            if sentence['sentence'][0] in numbers:
                if sentence['importance'] > 0.5:
                    sentence['importance'] = 2
                sentence['numberedListPart'] = True
            if sentence['sentence'][0] in listMarkers:
                if lastMarkedSentenceIndex:
                    for bmsIndex in range(lastMarkedSentenceIndex+1, sentenceIndex):
                        headerAndSentences['sentences'][bmsIndex]['betweenBullets'] = True
                betweenMarkers = True
                lastMarkedSentenceIndex = sentenceIndex
            if sentence['sentence'][0] in numbers:
                if lastMarkedSentenceIndex:
                    for bmsIndex in range(lastMarkedSentenceIndex+1, sentenceIndex):
                        headerAndSentences['sentences'][bmsIndex]['betweenNumbering'] = True
                betweenNumbering = True
                lastMarkedSentenceIndex = sentenceIndex
            if sentence['sentence'][-1] == ':':
                betweenMarkers = False
                betweenNumbering = False
                lastMarkedSentenceIndex = None
          
if __name__ == '__main__' and '__file__' not in globals():
#     print(listMarkers)
    pprint(headersAndSentencesByLevels)

### Вес предложения на определенном уровне иерархии: частотный признак.

Бъем предложения каждого куска текста на каждом уровне иерархии на токены.

In [ ]:
from pyrutok import Token, Sentence, Tokenizer, GraphemTag
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from collections import Counter
from pprint import pprint

morphAnalyzer = MorphAnalyzer()

headersAndWordsByLevels = {}
for level, headersAndSentences in headersAndSentencesByLevels.items():
    headersAndWordsByLevels[level] = []
    for headerAndSentences in headersAndSentences:
        words = Counter()
        for sentence in headerAndSentences['sentences']:
            tempText = sentence['sentence'].replace('*formula placeholder*', '')
            for us in unusualSymbols:
                tempText = tempText.replace(us, '')
            for sentence in Tokenizer(tempText):
                for token in sentence:
                    if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                        word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                        if word not in stopwords.words('russian'):
                            words[word] += 1
        headersAndWordsByLevels[level].append({
            'header': headerAndSentences['header'],
            'words': words
        })
 
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndWordsByLevels)

In [ ]:
import math

def countWordFrequencyFeature(word, 
                              wordCount, 
                              allWordsCount, 
                              textBlocksAtLevelCount, 
                              textBlocksWithWordAtLevelCount):
    return (wordCount / allWordsCount) * math.log2(textBlocksAtLevelCount * len(word) / textBlocksWithWordAtLevelCount)

def countWordsFrequencyFeature(requiredHeaderAndWords, headersAndWordsAtLevel):
    wordFrequencyWeights = Counter()
    for word in requiredHeaderAndWords['words']:
        wordFrequencyWeights[word] = countWordFrequencyFeature(
            word,
            requiredHeaderAndWords['words'][word],
            sum([requiredHeaderAndWords['words'][word] for word in requiredHeaderAndWords['words']]),
            len(headersAndWordsAtLevel),
            len([headerAndWordsAtLevel for headerAndWordsAtLevel in headersAndWordsAtLevel if word in headerAndWordsAtLevel['words']]))
    return wordFrequencyWeights

In [ ]:
from pprint import pprint

headersAndWordFrequencyWeightsByLevels = {}
for level, headersAndWords in headersAndWordsByLevels.items():
    headersAndWordFrequencyWeightsByLevels[level] = []
    for headerAndWords in headersAndWords:
        headersAndWordFrequencyWeightsByLevels[level].append({
            'header': headerAndWords['header'],
            'wordFrequencyWeights': countWordsFrequencyFeature(headerAndWords, headersAndWords)
        })
        
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndWordFrequencyWeightsByLevels)

In [ ]:
from pyrutok import Token, Sentence, Tokenizer, GraphemTag
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from pprint import pprint

def countSentenceFrequencyFeatureByLevels(requiredSentence, wordsFrequencyWeight):
    morphAnalyzer = MorphAnalyzer()
    sentenceWeight = 0
    for us in unusualSymbols:
        requiredSentence = requiredSentence.replace(us, '')
    for sentence in Tokenizer(requiredSentence):
        for token in sentence:
            if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                if word not in stopwords.words('russian'):
                    sentenceWeight += wordsFrequencyWeight[word]
    return sentenceWeight

In [ ]:
from pprint import pprint

for level, headersAndSentences in headersAndSentencesByLevels.items():
    for blockIndex in range(0, len(headersAndSentences)):
        for sentence in headersAndSentences[blockIndex]['sentences']:
            sentenceWeight = countSentenceFrequencyFeatureByLevels(
                sentence['sentence'], 
                headersAndWordFrequencyWeightsByLevels[level][blockIndex]['wordFrequencyWeights']
            )
            sentence['frequencyWeight'] = sentenceWeight # * sentence['importance']
    
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

### Вес предложения на определенном уровне иерархии: признак местоположения.

Cчитаем ту часть веса, которую относится к уровню, подразделов и т.д.

In [ ]:
def countSectionsPartWeight(headersAndTextHierarchy, requiredHeader):
    headerNumeration = requiredHeader.split(' ', -1)[0]
    weight = 1
    if set(headerNumeration).issubset(numbersAndPoint):
        nestedHierarchy = headersAndTextHierarchy
        for headerNumerationPart in headerNumeration.split('.'):
#             pprint(nestedHierarchy)
            weight *= min(
                int(headerNumerationPart),
                max(
                    int(level) for level in nestedHierarchy if level != ''
                ) - \
                int(headerNumerationPart) + 1 + \
                ((len(nestedHierarchy[''][0]) if isinstance(nestedHierarchy[''][0], list) else 1) 
                 if '' in nestedHierarchy 
                 else 0)
            )
            nestedHierarchy = nestedHierarchy[headerNumerationPart][1]
    else:
        if isinstance(headersAndTextHierarchy[''][0], list):
            for headerIndex in range(0, len(headersAndTextHierarchy[''][0])):
                if headersAndTextHierarchy[''][0][headerIndex] == requiredHeader:
                    weight *= min(
                        len(headersAndTextHierarchy[''][0]) - headerIndex,
                        max(
                            int(level) for level in headersAndTextHierarchy if level != ''
                        ) + headerIndex + 1
                    )
        else:
            weight *= min(
                1,
                max(
                    int(level) for level in headersAndTextHierarchy if level != ''
                ) + 1
            )
    return weight

In [ ]:
from pprint import pprint

for level, headersAndSentences in headersAndSentencesByLevels.items():
    if level == 0:
        continue
    for headerAndSentences in headersAndSentences:
        for sentcence in headerAndSentences['sentences']:
            headerAndSentences['sectionInverseWeight'] = countSectionsPartWeight(
                headersAndTextHierarchy, headerAndSentences['header']
            )
        
for level, headersAndSentences in headersAndSentencesByLevels.items():
    if level == 0:
        continue
    for headerAndSentences in headersAndSentences:
        for sentenceIndex in range(0, len(headerAndSentences['sentences'])):
            headerAndSentences['sentences'][sentenceIndex]['locationWeight'] = \
                1 / (
                min(sentenceIndex + 1, len(headerAndSentences['sentences']) - sentenceIndex) * \
                headerAndSentences['sectionInverseWeight']) # * \
#                  headerAndSentences['sentences'][sentenceIndex]['importance']
        
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

### Вес предложения на определенном уровне иерархии: заголовочный признак.

Определяем вышележащие заголовки.

In [ ]:
def determineOverlyingHeaders(headersAndTextHierarchy, requiredHeader):
    headerNumeration = requiredHeader.split(' ', -1)[0]
    overlyingHeaders = []
    overlyingHeaderNumeration = ''
    nestedHierarchy = headersAndTextHierarchy
    if set(headerNumeration).issubset(numbersAndPoint):
        for headerNumerationPart in headerNumeration.split('.')[:-1]:
            overlyingHeaders.append(overlyingHeaderNumeration + headerNumerationPart + ' ' + nestedHierarchy[headerNumerationPart][0]['header'])
            nestedHierarchy = nestedHierarchy[headerNumerationPart][1]
            overlyingHeaderNumeration += headerNumerationPart + '.'
            
    return overlyingHeaders

In [ ]:
from pyrutok import Token, Sentence, Tokenizer, GraphemTag
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

def countHeadingWeight(headersAndTextHierarchy, requiredHeader, overlyingHeaders, requiredSentence, wordFrequencyWeights):
    morphAnalyzer = MorphAnalyzer()
    headingWeight = 0
    
    requiredSentenceWords = set()
    for us in unusualSymbols:
        requiredSentence = requiredSentence.replace(us, '')
    for sentence in Tokenizer(requiredSentence):
        for token in sentence:
            if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                if word not in stopwords.words('russian'):
                    requiredSentenceWords.add(word)
                    
    if set(requiredHeader.split(' ', 1)[0]).issubset(numbersAndPoint):
        requiredHeader = requiredHeader.split(' ', 1)[1]
    
    for sentence in Tokenizer(requiredHeader):
        for token in sentence:
            if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                if word not in stopwords.words('russian') and \
                   word in requiredSentenceWords:
                    headingWeight += wordFrequencyWeights[word]
 
    nestedBlocksNumber = 1
    for overlyingHeader in overlyingHeaders[::-1]:
        headerNumeration = overlyingHeader.split(' ', -1)[0]
        nestedHierarchy = headersAndTextHierarchy
        for headerNumerationPart in headerNumeration.split('.'):
            nestedHierarchy = nestedHierarchy[headerNumerationPart][1]
        nestedBlocksNumber = len(nestedHierarchy) * nestedBlocksNumber
        for sentence in Tokenizer(overlyingHeader):
            for token in sentence:
                if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                    word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                    if word not in stopwords.words('russian') and \
                       word in requiredSentenceWords:
                        headingWeight += wordFrequencyWeights[word] / nestedBlocksNumber
                        
    return headingWeight

In [ ]:
from pprint import pprint

for level, headersAndSentences in headersAndSentencesByLevels.items():
    for headerAndSentences in headersAndSentences:
        headerAndSentences['overlyingHeaders'] = determineOverlyingHeaders(
            headersAndTextHierarchy, 
            headerAndSentences['header']
        )
        
for level, headersAndSentences in headersAndSentencesByLevels.items():
    for blockIndex in range(0, len(headersAndSentences)):
        for sentence in headersAndSentences[blockIndex]['sentences']:
            sentence['headingWeight'] = countHeadingWeight(
                headersAndTextHierarchy, 
                headersAndSentences[blockIndex]['header'], 
                headersAndSentences[blockIndex]['overlyingHeaders'], 
                sentence['sentence'], 
                headersAndWordFrequencyWeightsByLevels[level][blockIndex]['wordFrequencyWeights']
            ) # * sentence['importance']
        
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

### Вес предложения на определенном уровне иерархии: признак контрольных слов.

In [ ]:
commonControlWords1 = ['метод', 'задача', 'решение', 'функция', 'система', 'результат', 'модель', 'алгоритм', 'параметр', 'работа', 'схема', 'сетка', 'вектор', 'шаг', 'матрица', 'численный', 'слой', 'поверхность', 'процесс', 'качество', 'зависимость', 'аппроксимация', 'преобразование', 'моделирование', 'граничный', 'оператор', 'анализ', 'линейный', 'сеть', 'постановка', 'распределение', 'этап', 'обучение', 'проблема', 'эксперимент', 'построение', 'сравнение', 'интеграл', 'точность', 'признак', 'множество', 'оценка', 'интегральный', 'программа', 'разностный', 'набор', 'реализация', 'погрешность', 'нелинейный', 'интерполяция']
commonControlWords2 = ['задача', 'модель', 'функция', 'метод', 'алгоритм', 'система', 'работа', 'множество', 'решение', 'параметр', 'матрица', 'распределение', 'результат', 'образ', 'оценка', 'качество', 'уравнение', 'оптимальный', 'вектор', 'класс', 'объект', 'сеть', 'граф', 'набор', 'состояние', 'процесс', 'построение', 'шаг', 'схема', 'вероятность', 'элемент', 'выборка', 'анализ', 'признак', 'обучение', 'зависимость', 'сложность', 'вычисление', 'подход', 'эксперимент', 'ограничение', 'вход', 'реализация', 'свойство', 'этап', 'сравнение', 'дерево', 'оптимизация', 'итерация', 'проблема']
commonControlWords3 = ['работа', 'алгоритм', 'метод', 'задача', 'система', 'результат', 'модель', 'сеть', 'анализ', 'решение', 'качество', 'функция', 'признак', 'набор', 'оценка', 'обучение', 'программа', 'класс', 'подход', 'параметр', 'вектор', 'граф', 'реализовать', 'объект', 'процесс', 'построение', 'элемент', 'архитектура', 'модуль', 'эксперимент', 'сервер', 'тестирование', 'программный', 'разработать', 'выборка', 'матрица', 'слой', 'цель', 'обработка', 'зависимость', 'база', 'классификация', 'проблема', 'схема', 'характеристика', 'этап', 'тестовый', 'вывод', 'разработка', 'шаг']

In [ ]:
from pyrutok import Token, Sentence, Tokenizer, GraphemTag
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from pprint import pprint

def countSentenceControlFeatureByLevels(requiredSentence, wordsFrequencyWeight, tfIdfImportantWords):
    morphAnalyzer = MorphAnalyzer()
    sentenceWeight = 0
    for us in unusualSymbols:
        requiredSentence = requiredSentence.replace(us, '')
    for sentence in Tokenizer(requiredSentence):
        for token in sentence:
            if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                if word not in stopwords.words('russian'):
                    sentenceWeight += wordsFrequencyWeight[word] if word in commonControlWords2 else 0
    return sentenceWeight

In [ ]:
from pprint import pprint

for level, headersAndSentences in headersAndSentencesByLevels.items():
    for blockIndex in range(0, len(headersAndSentences)):
        for sentence in headersAndSentences[blockIndex]['sentences']:
            sentenceWeight = countSentenceControlFeatureByLevels(
                sentence['sentence'], 
                headersAndWordFrequencyWeightsByLevels[level][blockIndex]['wordFrequencyWeights'],
                tfIdfImportantWords
            )
            sentence['controlWeight'] = sentenceWeight # * sentence['importance']
    
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

### Извлекаем все веса, для всех разделов, чтобы потом их нормализовать, сложить и отсортировать.

In [ ]:
from pprint import pprint

for level, headersAndSentences in headersAndSentencesByLevels.items():
    if level == 0:
        continue
    for blockIndex in range(0, len(headersAndSentences)):
        headersAndSentences[blockIndex]['frequencyWeights'] = []
        headersAndSentences[blockIndex]['locationWeights'] = []
        headersAndSentences[blockIndex]['headingWeights'] = []
        headersAndSentences[blockIndex]['controlWeights'] = []
        for sentence in headersAndSentences[blockIndex]['sentences']:
            headersAndSentences[blockIndex]['frequencyWeights'].append(sentence['frequencyWeight'])
            headersAndSentences[blockIndex]['locationWeights'].append(sentence['locationWeight'])
            headersAndSentences[blockIndex]['headingWeights'].append(sentence['headingWeight'])
            headersAndSentences[blockIndex]['controlWeights'].append(sentence['controlWeight'])
   
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

### Нормализуем, складываем и сортируем веса предложений.

In [ ]:
from pprint import pprint

for level, headersAndSentences in headersAndSentencesByLevels.items():
    if level == 0:
        continue
    for blockIndex in range(0, len(headersAndSentences)):
        maxFrequencyWeight = max(headersAndSentences[blockIndex]['frequencyWeights']) if headersAndSentences[blockIndex]['frequencyWeights'] else 0
        maxFrequencyWeight = maxFrequencyWeight if maxFrequencyWeight != 0 else 1
        maxLocationWeight = max(headersAndSentences[blockIndex]['locationWeights']) if headersAndSentences[blockIndex]['locationWeights'] else 0
        maxLocationWeight = maxLocationWeight if maxLocationWeight != 0 else 1
        maxHeadingWeight = max(headersAndSentences[blockIndex]['headingWeights']) if headersAndSentences[blockIndex]['headingWeights'] else 0
        maxHeadingWeight = maxHeadingWeight if maxHeadingWeight != 0 else 1
        maxControlWeight = max(headersAndSentences[blockIndex]['controlWeights']) if headersAndSentences[blockIndex]['controlWeights'] else 0
        maxControlWeight = maxControlWeight if maxControlWeight != 0 else 1
        headersAndSentences[blockIndex]['weights'] = []
        for sentence in headersAndSentences[blockIndex]['sentences']:
            sentence['normalizedFrequencyWeight'] = sentence['frequencyWeight'] / maxFrequencyWeight
            sentence['normalizedLocationWeight'] = sentence['locationWeight'] / maxLocationWeight
            sentence['normalizedHeadingWeight'] = sentence['headingWeight'] / maxHeadingWeight
            sentence['normalizedControlWeight'] = sentence['controlWeight'] / maxControlWeight
            sentence['weight'] = (sentence['normalizedFrequencyWeight'] + \
                                  sentence['normalizedLocationWeight'] + \
                                  sentence['normalizedHeadingWeight'] + \
                                  sentence['normalizedControlWeight']) * \
                                 sentence['importance']
            headersAndSentences[blockIndex]['weights'].append(sentence['weight'])
#             del sentence['frequencyWeight']
#             del sentence['locationWeight']
#             del sentence['headingWeight']
#             del sentence['controlWeight']
#             del sentence['importance']
        headersAndSentences[blockIndex]['weights'].sort(reverse = True)
#         del headersAndSentences[blockIndex]['frequencyWeights']
#         del headersAndSentences[blockIndex]['locationWeights']
#         del headersAndSentences[blockIndex]['headingWeights']
#         del headersAndSentences[blockIndex]['controlWeights']
    
if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndSentencesByLevels)

### Выделяем по пять значимых предложений на раздел.

Этого достаточно с учетом длины предложений и возможного наличия картинок.

In [ ]:
from pprint import pprint

importantSentencesByLevels = {}
for level, headersAndSentences in headersAndSentencesByLevels.items():
    if level == 0:
        continue
    importantSentencesByLevels[level] = []
    for headerAndSentences in headersAndSentences:
        importantSentencesWeights = [weight for weight in headerAndSentences['weights'] if weight > 0][:5]
        importantSentences = []
        imageReferences = set()
        containsBulletList = False
        containsNumberedList = False
        for sentence in headerAndSentences['sentences']:
            if sentence['weight'] in importantSentencesWeights:
                importantSentences.append({
                    'sentence': sentence['sentence'],
                    'betweenBullets': sentence['betweenBullets'],
                    'bulletListPart': sentence['bulletListPart'],
                    'betweenNumbering': sentence['betweenNumbering'],
                    'numberedListPart': sentence['numberedListPart'],
                    'imageReferences': sentence['imageReferences'],
                    'tableReferences': sentence['tableReferences']
                })
                imageReferences = imageReferences.union(sentence['imageReferences'])
                if (sentence['bulletListPart']):
                    containsBulletList = True
                if (sentence['numberedListPart']):
                    containsNumberedList = True
        importantSentencesByLevels[level].append({
            'header': headerAndSentences['header'],
            'overlyingHeaders': headerAndSentences['overlyingHeaders'],
            'importantSentences': importantSentences,
            'sentencesImageReferences': imageReferences,
            'imageReferences':  headerAndSentences['imageReferences'] if 'imageReferences' in headerAndSentences else set(),
            'tableReferences': headerAndSentences['tableReferences'] if 'tableReferences' in headerAndSentences else set(),
            'containsBulletList': containsBulletList,
            'containsNumberedList': containsNumberedList
        })
     
if __name__ == '__main__' and '__file__' not in globals():
    pprint(importantSentencesByLevels)

### Оцениваем количество слайдов для каждого заголовка.

In [ ]:
# если в разделе есть таблица, то она получает отдельный слайд
# если в разделе есть рисунок и он единственный, то он получает отдельный слайд
# на каждый заголовок гарантированно приходится хотя бы один слайд
# изображение, ссылка на которое есть в тексте, получает отдельный слайд, если отношение его высоты к его ширине меньше 1.5 и больше 0.3
# изображение, ссылка на которое есть в тексте, получает отдельный слайд, если колиество изображений с одной подписью больше одного

for level, headersAndSentences in importantSentencesByLevels.items():
    for headerAndSentences in headersAndSentences:
        slidesCount = 1
        slidesCount += len(headerAndSentences['tableReferences'])
        slidesCount += 1 if len(headerAndSentences['imageReferences']) == 1 and \
                            len(headerAndSentences['imageReferences'].difference(headerAndSentences['sentencesImageReferences'])) == 1 \
                         else 0
        imageSlidesCount = 0
        for imageNumeration in headerAndSentences['sentencesImageReferences']:
            imageWithSameCaption = [caption for caption in captionsToImages
                                    if caption['numeration'] == imageNumeration]
            if len(imageWithSameCaption) > 1:
                imageSlidesCount += 1
            else:
                image = imageWithSameCaption[0]
                heightToWidthRatio = image['height'] / image['width']
                if heightToWidthRatio > 0.3 and heightToWidthRatio < 1.5:
                    imageSlidesCount += 1
        # это на случай, если в извлеченном тексте слишком много ссылок на изображения
        if len(headerAndSentences['sentencesImageReferences']) - imageSlidesCount > 1:
            imageSlidesCount = len(headerAndSentences['sentencesImageReferences']) - 1
        headerAndSentences['slidesCount'] = slidesCount + imageSlidesCount
        
if __name__ == '__main__' and '__file__' not in globals():
    pprint(importantSentencesByLevels)

### Определяем важность заголовка по наличию или отсутствию в нем ключевых слов. Оцениваем количество слайдов для каждого уровня важности.

In [ ]:
importantHeadersMarkers = ['результат', 'введение', 'вывод', 'метод', 'подход', 'заключение']
importantHeadersBigrams = ['постановка задача']

unimportantHeadersMarkers = ['обзор', 'приложение', 'литература']
unimportantHeadersBigrams = ['список литература']

In [ ]:
from pyrutok import Token, Sentence, Tokenizer, GraphemTag
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from pprint import pprint

absolutelyImportantHeaders = []
absolutelyImportantSlidesCount = 0
highImportantHeaders = []
highImportantSlidesCount = 0
mediumImportantHeaders = []
mediumImportantSlidesCount = 0
tableReferences = set()
for level, headersAndSentences in importantSentencesByLevels.items():
    if level == 0:
        continue
    for headerAndSentences in headersAndSentences:
        header = headerAndSentences['header']
        headerWords = []
        for sentence in Tokenizer(header):
            for token in sentence:
                if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                    word = morphAnalyzer.parse(token.get_escaped_data())[0].normal_form
                    if word not in stopwords.words('russian'):
                        headerWords.append(word)
        headerBigrams = [headerWords[i] + ' ' + headerWords[i+1] for i in range(0, len(headerWords) - 1)]
#         print(headerWords)
        if ('приложение' in headerWords) or \
           ('литература' in headerWords and len(headerWords) == 1) or \
           ('список литература' in headerBigrams and len(headerBigrams) == 1) or \
           'алгоритм' in headerWords:
            # разделы, которые совсем не нужны
            pass
        elif ('введение' in headerWords or 'заключение' in headerWords or 'вывод' in headerWords) or \
             'постановка задача' in headerBigrams:
#              headerAndSentences['containsBulletList'] or headerAndSentences['containsNumberedList']:
            tableSlides = len(headerAndSentences['tableReferences'].intersection(tableReferences))
            tableReferences = tableReferences.union(headerAndSentences['tableReferences'])
            absolutelyImportantHeaders.append(header)
            absolutelyImportantSlidesCount += headerAndSentences['slidesCount'] - tableSlides
            highImportantHeaders.append(header)
            highImportantSlidesCount += headerAndSentences['slidesCount'] - tableSlides
            mediumImportantHeaders.append(header)
            mediumImportantSlidesCount += headerAndSentences['slidesCount'] - tableSlides
        elif 'результат' in headerWords or 'метод' in headerWords or \
             'подход' in headerWords or 'система' in headerWords or \
             'метрика' in headerWords or 'реализация' in headerWords or \
             headerAndSentences['tableReferences']:
#             print(headerAndSentences['header'], headerAndSentences['tableReferences'])
            tableSlides = len(headerAndSentences['tableReferences'].intersection(tableReferences))
            tableReferences = tableReferences.union(headerAndSentences['tableReferences'])
            highImportantHeaders.append(header)
            highImportantSlidesCount += headerAndSentences['slidesCount'] - tableSlides
            mediumImportantHeaders.append(header)
            mediumImportantSlidesCount += headerAndSentences['slidesCount'] - tableSlides
        elif 'обзор' in headerWords or \
             ('приложение' in headerWords and len(headerWords) == 1) or \
             ('литература' in headerWords and len(headerWords) == 1) or \
             ('список литература' in headerBigrams and len(headerBigrams) == 1):
            # разделы, которые совсем не нужны
            pass
        else:
            tableSlides = len(headerAndSentences['tableReferences'].intersection(tableReferences))
            tableReferences = tableReferences.union(headerAndSentences['tableReferences'])
            mediumImportantHeaders.append(header)
            mediumImportantSlidesCount += headerAndSentences['slidesCount'] - tableSlides
            
if __name__ == '__main__' and '__file__' not in globals():
    print(absolutelyImportantHeaders)
    print(absolutelyImportantSlidesCount)
    print(highImportantHeaders)
    print(highImportantSlidesCount)
    print(mediumImportantHeaders)
    print(mediumImportantSlidesCount)

In [ ]:
presentationSlidesCount = 0
presentationHeaders = []
if absolutelyImportantSlidesCount > 10 and absolutelyImportantSlidesCount <= 16:
    presentationSlidesCount = absolutelyImportantSlidesCount
    presentationHeaders = absolutelyImportantHeaders
if highImportantSlidesCount > 10 and highImportantSlidesCount <= 16:
    presentationSlidesCount = highImportantSlidesCount
    presentationHeaders = highImportantHeaders
if mediumImportantSlidesCount > 10 and mediumImportantSlidesCount <= 16:
    presentationSlidesCount = mediumImportantSlidesCount
    presentationHeaders = mediumImportantHeaders
    
presentationContent = []
for level, headersAndSentences in importantSentencesByLevels.items():
    for headerAndSentences in headersAndSentences:
        if headerAndSentences['header'] in presentationHeaders:
            if set(headerAndSentences['header'].split(' ', 1)[0]).issubset(numbersAndPoint): 
                header = headerAndSentences['header'].split(' ', 1)[1]
            else:
                header = headerAndSentences['header']
            presentationContent.append({
                'header': headerAndSentences['header'],
                'bareHeader': header,
                'importantSentences': headerAndSentences['importantSentences'],
                'sectionImageReferences': headerAndSentences['imageReferences']
                                          if len(headerAndSentences['imageReferences']) == 1 and \
                                             len(headerAndSentences['imageReferences'].difference(headerAndSentences['sentencesImageReferences'])) == 1
                                          else set(),
                'tableReferences': headerAndSentences['tableReferences']
            })
            
presentationContent = sorted(presentationContent, key = lambda h: headersByOrder.index(h['header']))
            
if __name__ == '__main__' and '__file__' not in globals():
    pprint(presentationContent)